In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

## DGP

In [2]:
date = pd.to_datetime(["2020-01-01", "2021-01-01"])
units = np.array(range(1, 500+1))

region = np.array(["S", "N", "E", "W"])
reg_ps = dict(zip(region,    [0.2, 0.4, 0.6, 0.8]))
reg_fe = dict(zip(region,    [5, 3, 1, -1]))
reg_trend = dict(zip(region, [-1, 3, 6, 7]))

np.random.seed(1)
unit_region = pd.Series(np.random.choice(region, len(units)))
treated_unit = np.random.binomial(1, unit_region.map(reg_ps))

df = pd.DataFrame(dict(
    date = np.tile(date, len(units)),
    unit = np.repeat(units, len(date)),
    region = np.repeat(unit_region, len(date)),
    treated_unit = np.repeat(treated_unit, len(date)),
    
    unit_fe = np.repeat(np.random.normal(0, 2, size=len(units)), len(date)),
    time_fe = np.tile(np.random.normal(size=len(date)), len(units)),
    
    reg_ps = pd.Series(np.repeat(unit_region, len(date))).map(reg_ps),
    reg_fe = pd.Series(np.repeat(unit_region, len(date))).map(reg_fe),
    reg_trend = pd.Series(np.repeat(unit_region, len(date))).map(reg_trend))
).assign(
    treated = lambda d: (d["treated_unit"] == 1).astype(int),
    post = lambda d: (d["date"] >= "2021-01-01").astype(int)
).assign(
    y0 = lambda d: (10
                    + d["unit_fe"] 
                    + d["time_fe"] 
                    + d["reg_fe"]
                    + d["reg_trend"]*(d["date"] >= "2021-01-01")),
).assign(
    y1 = lambda d: d["y0"] + 1
).assign(
    tau = lambda d: d["y1"] - d["y0"],
    y = lambda d: np.where(d["treated"]*d["post"] == 1, d["y1"], d["y0"]) + np.random.normal(0,1,len(d))
)

df.head()

,date,unit,region,treated_unit,unit_fe,time_fe,reg_ps,reg_fe,reg_trend,treated,post,y0,y1,tau,y
0,2020-01-01,1,N,0,-2.226872,0.288617,0.4,3,3,0,0,11.061745,12.061745,1.0,12.693680
0,2021-01-01,1,N,0,-2.226872,0.058307,0.4,3,3,0,1,13.831435,14.831435,1.0,13.429646
1,2020-01-01,2,W,0,-0.134820,0.288617,0.8,-1,7,0,0,9.153797,10.153797,1.0,8.953857
1,2021-01-01,2,W,0,-0.134820,0.058307,0.8,-1,7,0,1,15.923487,16.923487,1.0,15.930876
2,2020-01-01,3,S,0,2.322880,0.288617,0.2,5,-1,0,0,17.611497,18.611497,1.0,17.887161


## Traditional DiD With Covariates

In [3]:
smf.ols("y~post*treated", data=df).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,13.4804,0.170,79.434,0.000,13.147,13.813
post,2.0901,0.240,8.709,0.000,1.619,2.561
treated,-2.1725,0.241,-8.998,0.000,-2.646,-1.699
post:treated,3.6291,0.341,10.628,0.000,2.959,4.299


In [4]:
smf.ols("y~post*treated + C(region)", data=df).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,13.1802,0.232,56.871,0.000,12.725,13.635
C(region)[T.N],0.3098,0.242,1.281,0.201,-0.165,0.785
C(region)[T.S],0.6852,0.243,2.823,0.005,0.209,1.162
C(region)[T.W],-0.9844,0.246,-3.994,0.000,-1.468,-0.501
post,2.0901,0.235,8.886,0.000,1.629,2.552
treated,-1.6090,0.252,-6.389,0.000,-2.103,-1.115
post:treated,3.6291,0.335,10.844,0.000,2.972,4.286


In [5]:
smf.ols("y~post*(treated + C(region))", data=df).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,11.3287,0.239,47.379,0.000,10.859,11.798
C(region)[T.N],1.7889,0.290,6.175,0.000,1.220,2.357
C(region)[T.S],4.1519,0.291,14.287,0.000,3.582,4.722
C(region)[T.W],-1.4337,0.295,-4.858,0.000,-2.013,-0.855
post,5.7932,0.338,17.132,0.000,5.130,6.457
post:C(region)[T.N],-2.9581,0.410,-7.220,0.000,-3.762,-2.154
post:C(region)[T.S],-6.9333,0.411,-16.870,0.000,-7.740,-6.127
post:C(region)[T.W],0.8986,0.417,2.153,0.032,0.080,1.718
treated,-0.2784,0.225,-1.235,0.217,-0.721,0.164
post:treated,0.9678,0.319,3.036,0.002,0.342,1.593


In [6]:
for region in df["region"].unique():
    att = smf.ols("y~post*treated", data=df.query("region==@region")).fit().params["post:treated"]
    print(region, att)
    

N 0.9226567466899882
W 1.1623121008046138
S 0.9169824580119377
E 0.9421142952058243


## Doubly Robust DiD



$$
\hat{\tau}_{DRDID} = \hat{\Delta y}_1^{DR} - \hat{\Delta y}_0^{DR}
$$


$$
\hat{\Delta y}_1^{DR} = 1/N_{tr}\sum_{i \in tr} \big(\Delta y - \hat{m}(X)\big)
$$


$$
w_{co} = \hat{e}(X)\dfrac{1}{1-\hat{e}(X)}
$$


$$
\hat{\Delta y}_0^{DR} = \sum_{i \in co} w_{co} \big(\Delta y - \hat{m}(X)\big) \big/ \sum w_{co}
$$


### Fit Propensity Score Model

In [7]:
ps_model = smf.logit("treated~C(region)",
                     data=df.query("post==0")
                    ).fit()

ps_df = df.assign(ps=ps_model.predict(df))
ps_df.head()

Optimization terminated successfully.
         Current function value: 0.580103
         Iterations 6


,date,unit,region,treated_unit,unit_fe,time_fe,reg_ps,reg_fe,reg_trend,treated,post,y0,y1,tau,y,ps
0,2020-01-01,1,N,0,-2.226872,0.288617,0.4,3,3,0,0,11.061745,12.061745,1.0,12.693680,0.409836
0,2021-01-01,1,N,0,-2.226872,0.058307,0.4,3,3,0,1,13.831435,14.831435,1.0,13.429646,0.409836
1,2020-01-01,2,W,0,-0.134820,0.288617,0.8,-1,7,0,0,9.153797,10.153797,1.0,8.953857,0.836066
1,2021-01-01,2,W,0,-0.134820,0.058307,0.8,-1,7,0,1,15.923487,16.923487,1.0,15.930876,0.836066
2,2020-01-01,3,S,0,2.322880,0.288617,0.2,5,-1,0,0,17.611497,18.611497,1.0,17.887161,0.213235


### Outcome Model

In [8]:
delta_y = df.set_index("unit").query("post==1")["y"]  - df.set_index("unit").query("post==0")["y"]

df_delta_y = ps_df.set_index("unit").join(delta_y.rename("delta_y"))

df_delta_y.head()

,date,region,treated_unit,unit_fe,time_fe,reg_ps,reg_fe,reg_trend,treated,post,y0,y1,tau,y,ps,delta_y
unit,,,,,,,,,,,,,,,,
1,2020-01-01,N,0,-2.226872,0.288617,0.4,3,3,0,0,11.061745,12.061745,1.0,12.693680,0.409836,0.735966
1,2021-01-01,N,0,-2.226872,0.058307,0.4,3,3,0,1,13.831435,14.831435,1.0,13.429646,0.409836,0.735966
2,2020-01-01,W,0,-0.134820,0.288617,0.8,-1,7,0,0,9.153797,10.153797,1.0,8.953857,0.836066,6.977019
2,2021-01-01,W,0,-0.134820,0.058307,0.8,-1,7,0,1,15.923487,16.923487,1.0,15.930876,0.836066,6.977019
3,2020-01-01,S,0,2.322880,0.288617,0.2,5,-1,0,0,17.611497,18.611497,1.0,17.887161,0.213235,-3.269223


In [9]:
outcome_model = smf.ols("delta_y ~ C(region)",
                        data=df_delta_y.query("post==0 & treated==0")).fit()

df_dr = df_delta_y.assign(y_hat = outcome_model.predict(df_delta_y))

df_dr.head()

,date,region,treated_unit,unit_fe,time_fe,reg_ps,reg_fe,reg_trend,treated,post,y0,y1,tau,y,ps,delta_y,y_hat
unit,,,,,,,,,,,,,,,,,
1,2020-01-01,N,0,-2.226872,0.288617,0.4,3,3,0,0,11.061745,12.061745,1.0,12.693680,0.409836,0.735966,2.853578
1,2021-01-01,N,0,-2.226872,0.058307,0.4,3,3,0,1,13.831435,14.831435,1.0,13.429646,0.409836,0.735966,2.853578
2,2020-01-01,W,0,-0.134820,0.288617,0.8,-1,7,0,0,9.153797,10.153797,1.0,8.953857,0.836066,6.977019,6.529162
2,2021-01-01,W,0,-0.134820,0.058307,0.8,-1,7,0,1,15.923487,16.923487,1.0,15.930876,0.836066,6.977019,6.529162
3,2020-01-01,S,0,2.322880,0.288617,0.2,5,-1,0,0,17.611497,18.611497,1.0,17.887161,0.213235,-3.269223,-1.129266


In [10]:
tr = df_dr.query("treated==1")
co = df_dr.query("treated==0")

dy1_treat = (tr["delta_y"] - tr["y_hat"]).mean()

w_cont = co["ps"]/(1-co["ps"])
dy0_treat = np.average(co["delta_y"] - co["y_hat"], weights=w_cont)

print("ATT:", dy1_treat - dy0_treat)

ATT: 1.0261567060020285


In [11]:
df_dr.query("post==0 & treated==1")["tau"].mean()

1.0

## Messing Up the PS model

In [12]:
np.random.seed(123)

df_dr_ps_messed = df_dr.assign(ps=np.random.uniform(.3, .7, len(df_dr)))

tr = df_dr_ps_messed.query("treated==1")
co = df_dr_ps_messed.query("treated==0")

dy1_treat = (tr["delta_y"] - tr["y_hat"]).mean()

w_cont = co["ps"]/(1-co["ps"])
dy0_treat = np.average(co["delta_y"] - co["y_hat"], weights=w_cont)

print("ATT:", dy1_treat - dy0_treat)

ATT: 0.9601085810600898


## Messed up Outcome Model

In [13]:
np.random.seed(123)

df_dr_outcome_messed = df_dr.assign(y_hat=np.random.uniform(0, 1, len(df_dr)))

tr = df_dr_outcome_messed.query("treated==1")
co = df_dr_outcome_messed.query("treated==0")

dy1_treat = (tr["delta_y"] - tr["y_hat"]).mean()

w_cont = co["ps"]/(1-co["ps"])
dy0_treat = np.average(co["delta_y"] - co["y_hat"], weights=w_cont)

print("ATT:", dy1_treat - dy0_treat)

ATT: 1.063554541645658
